# Titanic: Machine Learning from disaster

## Competition Description

The sinking of the RMS Titanic is one of the most infamous shipwrecks in history.  On April 15, 1912, during her maiden voyage, the Titanic sank after colliding with an iceberg, killing 1502 out of 2224 passengers and crew. This sensational tragedy shocked the international community and led to better safety regulations for ships.

One of the reasons that the shipwreck led to such loss of life was that there were not enough lifeboats for the passengers and crew. Although there was some element of luck involved in surviving the sinking, some groups of people were more likely to survive than others, such as women, children, and the upper-class.

In this challenge, we ask you to complete the analysis of what sorts of people were likely to survive. In particular, we ask you to apply the tools of machine learning to predict which passengers survived the tragedy.

## Goal
It is your job to predict if a passenger survived the sinking of the Titanic or not. 
For each PassengerId in the test set, you must predict a 0 or 1 value for the Survived variable.

## Metric
Your score is the percentage of passengers you correctly predict. This is known simply as "accuracy”.

## Submission File Format
You should submit a csv file with exactly 418 entries plus a header row. Your submission will show an error if you have extra columns (beyond PassengerId and Survived) or rows.

The file should have exactly 2 columns:

PassengerId (sorted in any order)
Survived (contains your binary predictions: 1 for survived, 0 for deceased)

# Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

# Load data

In [ ]:
train_df = pd.read_csv("../input/train.csv")
train_df.head()

In [ ]:
pred_df = pd.read_csv("../input/test.csv")

# append the two datasets for feature engineering
train_df["dataset"] = "train"
pred_df["dataset"] = "pred"
data_df = train_df.append(pred_df, sort=True)

In [ ]:
# show missing values
data_df.isnull().sum()

# Feature engineering

The 'Name' column contains interesting information: the family name and the title. The title can give us relevant information about the marital status and age. The family name can help us to find if some persons of the individual's family survived.

In [ ]:
# clean name and extracting Title

data_df['Title'] = data_df['Name'].str.extract('([A-Za-z]+)\.', expand=True)

# replace rare titles with more common ones
mapping = {'Mlle': 'Miss', 'Major': 'Mr', 'Col': 'Mr', 'Sir': 'Mr', 'Don': 'Mr', 'Mme': 'Miss',
          'Jonkheer': 'Mr', 'Lady': 'Mrs', 'Capt': 'Mr', 'Countess': 'Mrs', 'Ms': 'Miss', 'Dona': 'Mrs'}

data_df.replace({'Title': mapping}, inplace=True)

A lot of age values are missing. We can impute the using the median of the persons with the same title.

In [ ]:
titles = list(data_df.Title.value_counts().index)

# for each title, impute missing age by the median of the persons with the same title
for title in titles:
    age_to_impute = data_df.groupby('Title')['Age'].median()[title]
    data_df.loc[(data_df['Age'].isnull()) & (data_df['Title'] == title), 'Age'] = age_to_impute

The Parch variable corresponds to the number of parents or children aboard the Titanic. The SibSp variable corresponds to the number of siblings or spouses aboard the Titanic. We can add those two variables in order to get the size of the family that was onboard.

In [ ]:
# compute family size
data_df['Family_Size'] = data_df['Parch'] + data_df['SibSp']

Now we can try to retrieve information about family survival.
There are two ways to find people from the same family:
- they have the same last name and they paid the same family fare
- they have the same ticket id

In [ ]:
# get the last name (family name): the string part before the ,
data_df['Last_Name'] = data_df['Name'].apply(lambda x: str.split(x, ",")[0])

# remove null fare values
data_df['Fare'].fillna(data_df['Fare'].mean(), inplace=True)

In [ ]:
# get information about family survival using Last_Name and Fare

# 0.5: default value for no information
# 1: someone of the same family survived
# 0: we don't know if somebody survived but we know that somebody died

default_survival_value = 0.5
data_df['Family_Survival'] = default_survival_value

for grp, grp_df in data_df.groupby(['Last_Name', 'Fare']):
    # if a family group is found
    if (len(grp_df) != 1):
        # for every person, look for the other people from the same family
        for ind, row in grp_df.iterrows():
            smax = grp_df.drop(ind)['Survived'].max()
            smin = grp_df.drop(ind)['Survived'].min()
            passID = row['PassengerId']
            
            if (smax == 1.0): # if anyone in the family survived, assign 1
                data_df.loc[data_df['PassengerId'] == passID, 'Family_Survival'] = 1
            elif (smin==0.0): # else if we saw someone dead, assign 0
                data_df.loc[data_df['PassengerId'] == passID, 'Family_Survival'] = 0

print("Number of passengers with family survival information:", 
      data_df.loc[data_df['Family_Survival']!=0.5].shape[0])

In [ ]:
# get information about family survival using the Ticket number

for _, grp_df in data_df.groupby('Ticket'):
    # if a family group is found
    if (len(grp_df) != 1):
        # for every person, look for the other people from the same family
        for ind, row in grp_df.iterrows():
            if (row['Family_Survival'] == 0) | (row['Family_Survival']== 0.5):
                smax = grp_df.drop(ind)['Survived'].max()
                smin = grp_df.drop(ind)['Survived'].min()
                passID = row['PassengerId']
                if (smax == 1.0): # if anyone in the family survived, assign 1
                    data_df.loc[data_df['PassengerId'] == passID, 'Family_Survival'] = 1
                elif (smin==0.0): # else if we saw someone dead, assign 0
                    data_df.loc[data_df['PassengerId'] == passID, 'Family_Survival'] = 0
                        
print("Number of passenger with family survival information: " 
      +str(data_df[data_df['Family_Survival']!=0.5].shape[0]))

Next we regroup Fare and Age into bins:

In [ ]:
# fare bins
data_df['FareBin'] = pd.qcut(data_df['Fare'], 5)
label = LabelEncoder()
data_df['FareBin_Code'] = label.fit_transform(data_df['FareBin'])
data_df.drop(['Fare'], 1, inplace=True)

In [ ]:
# age bins
data_df['AgeBin'] = pd.qcut(data_df['Age'], 4)
label = LabelEncoder()
data_df['AgeBin_Code'] = label.fit_transform(data_df['AgeBin'])
data_df.drop(['Age'], 1, inplace=True)

In [ ]:
# encode sex variable
data_df['Sex'].replace(['male','female'],[0,1],inplace=True)

In [ ]:
# choose features and labels
label = "Survived"
features = ["Pclass", "Sex", "Family_Size", "Family_Survival", "FareBin_Code", "AgeBin_Code"]

# split back data_df into train and prediction sets
train_df = data_df[data_df["dataset"] == "train"][features + [label]]
pred_df = data_df[data_df["dataset"] == "pred"][features]

# convert Survived variable to int for train dataset
train_df["Survived"] = train_df["Survived"].astype(np.int64)

In [ ]:
train_df.head()

# Train and test KNN classifiers

In [ ]:
# setup dataframes
X = train_df[features]
y = train_df['Survived']
X_pred = pred_df

# scale data for KNN classifier
std_scaler = StandardScaler()
X = std_scaler.fit_transform(X)
X_pred = std_scaler.transform(X_pred)

In [ ]:
# setup parameters values for grid search
n_neighbors = [6,7,8,9,10,11,12,14,16,18,20,22]
weights = ['uniform', 'distance']
leaf_size = list(range(1,50,5))
hyperparams = {'weights': weights, 'leaf_size': leaf_size, 'n_neighbors': n_neighbors}


gd=GridSearchCV(estimator = KNeighborsClassifier(), param_grid = hyperparams, verbose=True, 
                cv=10, scoring = "roc_auc")
gd.fit(X, y)
print(gd.best_score_)
print(gd.best_estimator_)

# Submit predictions

In [ ]:
# make predictions
gd.best_estimator_.fit(X, y)
y_pred = gd.best_estimator_.predict(X_pred)

# output predictions dataframe
temp = pd.DataFrame(pd.read_csv("../input/test.csv")['PassengerId'])
temp['Survived'] = y_pred
temp.to_csv("../working/submission.csv", index = False)